In [118]:
from bs4 import BeautifulSoup
import pandas
import numpy as np
import pandas as pd
import csv
import requests
import re
import pprint

#     """
#     beautiful soup parses through the user's HTML file and gets each 
#     link's article's body text
#     """


def get_articles_table(input_file, table_display):
    with open(input_file) as fp:
        soup = BeautifulSoup(fp, "lxml")

    all_articles_dictionary = {}
    pp = pprint.PrettyPrinter(indent=4)
    article_id = 0

    for link in soup.find_all("div", { "class" : "_2pin" }):
        article_id+=1
    #     pp.pprint(str(link.div.div.div.div.div))
        single_article_dictionary = {}
        post_section = 0

        for string in link.div.div.div.div.div:
            post_section+=1
            if post_section is 1:
                single_article_dictionary["hyperlink"] = str(string)[5:len(string)-7]
            if post_section is 2:
                single_article_dictionary["title"] = str(string)[5:len(string)-7]
            if post_section is 3:
                single_article_dictionary["publisher"] = str(string)[5:len(string)-7]
        all_articles_dictionary[article_id] = single_article_dictionary

    pd_articles = pd.DataFrame(all_articles_dictionary)
    if table_display:
        display(pd_articles)
        


    #         print(link.div.div.div.div.div.div.get_text())
    #         print(link.div.div.div.div.div)
    #     dictionary[link.div.div.div.div.div.div.get_text()]
    # article_descriptions.head()
    


In [119]:
get_articles_table("your_saved_items.html",True)


,1,2,3,4,5,6,7,8,9,10,...,992,993,994,995,996,997,998,999,1000,1001
hyperlink,https://www.youtube.com/watch?v=cU1LhcEh8Ms,http://www.kurdistan24.net/en/news/13cf816e-8e...,https://www.thisisinsider.com/nobel-prize-winn...,http://www.theatlantic.com/entertainment/archi...,https://liberationschool.org/ch-6-the-supreme-...,https://www.cnbc.com/2018/10/03/amazon-hourly-...,https://gritpost.com/corporate-profits-america...,http://www.theatlantic.com/health/archive/2015...,https://www.thedailybeast.com/kavanaugh-has-be...,https://www.visitthedprk.org/5-surprising-thin...,...,http://www.theatlantic.com/notes/all/2015/08/d...,https://www.vice.com/en_us/article/avykp8/what...,https://dancingastronaut.com/2015/07/mat-zos-l...,https://medium.com/@aristoorginos/social-justi...,https://www.attackmagazine.com/technique/tutor...,http://www.theatlantic.com/health/archive/2014...,http://www.theatlantic.com/politics/archive/20...,https://jacobinmag.com/2015/07/hogeland-indepe...,http://www.theatlantic.com/health/archive/2014...,https://ask.audio/articles/10-logic-pro-x-key-...
publisher,www.youtube.com,www.kurdistan24.net,www.thisisinsider.com,www.theatlantic.com,liberationschool.org,www.cnbc.com,gritpost.com,www.theatlantic.com,www.thedailybeast.com,www.visitthedprk.org,...,www.theatlantic.com,www.vice.com,dancingastronaut.com,medium.com,www.attackmagazine.com,www.theatlantic.com,www.theatlantic.com,jacobinmag.com,www.theatlantic.com,ask.audio
title,Jordan Peterson doesn't understand postmodernism,Noam Chomsky says US should stay in Syria to p...,"A Nobel Prize winner sold his medal for $765,0...",Starship Troopers: One of the Most Misundersto...,The Supreme Court: last line of defense for th...,Amazon's hourly workers lose monthly bonuses a...,"Corporate Profits Hit Record High, and 40% of ...","Childhood Guilt, Adult Depression?",Kavanaugh Has Become a Hero to the Incel Commu...,5 surprising things that North Korea and Irela...,...,Debating the New Political Correctness on Campus,What Black Anime Fans Can Teach Us About Race ...,Mat Zo's long-awaited remix of Arty's 'Stronge...,Social Justice Bullies: The Authoritarianism o...,Low-End Theory: Exploring Eight Common Basslin...,The Dark Side of Emotional Intelligence,Can Religion and Science Coexist?,Not Our Independence Day,Living Myths About Virginity,10 Logic Pro X Key Commands That Will Rock You...


In [ ]:
for link in soup.find_all("div", { "class" : "_3-96" }):
    article_id+=1
    
    if(link.div !=None):
        pp.pprint(str(link))
    single_article_dictionary = {}
    post_section = 0
    
#     if(link.div !=None):
        
#     for string in link.div.div.div.div.div.div:
#         post_section+=1
#         if post_section is 1:
#             single_article_dictionary["hyperlink"] = str(string)[5:len(string)-7]
#         if post_section is 2:
#             single_article_dictionary["title"] = str(string)[5:len(string)-7]
#         if post_section is 3:
#             single_article_dictionary["publisher"] = str(string)[5:len(string)-7]

#     all_articles_dictionary[article_id] = single_article_dictionary

# pd_articles = pd.DataFrame(all_articles_dictionary)
# pd_articles
